In [2]:
import pandas as pd

In [3]:
from pathlib import Path

path1 = Path("../data/item_properties_part1.csv")
path2 = Path("../data/item_properties_part2.csv")

In [4]:
path = '../data/events.csv'


In [5]:
def load_item_categories(path1, path2):
    """
    Load and process item category mappings.
    
    Args:
        path1: Path to first item properties CSV
        path2: Path to second item properties CSV
    
    Returns:
        DataFrame with columns: itemid, category_id
    """
    p1 = pd.read_csv(path1)
    p2 = pd.read_csv(path2)

    items = pd.concat([p1, p2], ignore_index=True)
    cats = items[items["property"] == "categoryid"]

    cats = cats[["itemid", "value"]].drop_duplicates()
    cats = cats.rename(columns={"value": "category_id"})

    return cats

In [6]:
cat_df=load_item_categories(path1, path2)

In [7]:
def load_and_clean_events(path, cat_df, is_train=True, train_stats=None):
    """
    Load and preprocess event data with feature engineering.
    
    Args:
        path: Path to CSV file containing events
        cat_df: DataFrame with item->category mappings
        is_train: If True, compute training statistics. If False, use provided stats.
        train_stats: Dictionary of precomputed statistics (required if is_train=False)
    
    Returns:
        If is_train=True: (processed_df, train_stats)
        If is_train=False: processed_df
    """
    df = pd.read_csv(path)

    # Handle timestamp conversion properly
    if df["timestamp"].dtype == 'object':
        # Already datetime string from CSV
        df["timestamp"] = pd.to_datetime(df["timestamp"])
    else:
        # Raw milliseconds
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    
    # Drop any invalid timestamps
    df = df.dropna(subset=["timestamp"])

    # Temporal features
    df["hour"] = df["timestamp"].dt.hour
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    df["is_weekend"] = (df["dayofweek"] >= 5).astype(int)
    df["is_evening"] = ((df["hour"] >= 18) & (df["hour"] <= 23)).astype(int)
    df["is_working_hours"] = ((df["hour"] >= 9) & (df["hour"] <= 17)).astype(int)

    # Create target variable
    df["label"] = (df["event"] == "transaction").astype(int)

    # Merge category information
    df = df.merge(cat_df, on="itemid", how="left")
    df["category_id"] = df["category_id"].fillna(-1).astype(int)

    # Compute training statistics
    if is_train:
        train_stats = {}

        # Item popularity (total interactions per item)
        train_stats["item_popularity"] = (
            df.groupby("itemid").size().reset_index(name="item_popularity")
        )

        # Item conversion rate (purchases / total interactions)
        item_conv = df.groupby("itemid")["label"].agg(["sum", "count"]).reset_index()
        item_conv["item_conversion_rate"] = item_conv["sum"] / item_conv["count"]
        train_stats["item_conversion_rate"] = item_conv[
            ["itemid", "item_conversion_rate"]
        ]

        # Category popularity
        train_stats["category_popularity"] = (
            df.groupby("category_id").size().reset_index(name="category_popularity")
        )

        # Category conversion rate
        cat_conv = df.groupby("category_id")["label"].agg(["sum", "count"]).reset_index()
        cat_conv["category_conversion_rate"] = cat_conv["sum"] / cat_conv["count"]
        train_stats["category_conversion_rate"] = cat_conv[
            ["category_id", "category_conversion_rate"]
        ]

        # User activity level
        user_activity = (
            df.groupby("visitorid").size().reset_index(name="user_event_count")
        )
        train_stats["user_activity"] = user_activity

        # User purchase rate
        user_conv = df.groupby("visitorid")["label"].agg(["sum", "count"]).reset_index()
        user_conv["user_purchase_rate"] = user_conv["sum"] / user_conv["count"]
        train_stats["user_purchase_rate"] = user_conv[
            ["visitorid", "user_purchase_rate"]
        ]

        # Recent item popularity (last 7 days)
        max_ts = df["timestamp"].max()
        recent_cutoff = max_ts - pd.Timedelta(days=7)
        recent_df = df[df["timestamp"] >= recent_cutoff]
        train_stats["recent_item_popularity"] = (
            recent_df.groupby("itemid")
            .size()
            .reset_index(name="recent_item_popularity")
        )

        # Item popularity for ratio calculation
        train_stats["item_popularity_for_ratio"] = train_stats["item_popularity"]

        # Ensure all required stats exist (for edge cases)
        required = {
            "item_popularity": ["itemid", "item_popularity"],
            "item_conversion_rate": ["itemid", "item_conversion_rate"],
            "category_popularity": ["category_id", "category_popularity"],
            "category_conversion_rate": ["category_id", "category_conversion_rate"],
            "user_activity": ["visitorid", "user_event_count"],
            "user_purchase_rate": ["visitorid", "user_purchase_rate"],
            "recent_item_popularity": ["itemid", "recent_item_popularity"],
        }

        for key, cols in required.items():
            if key not in train_stats:
                train_stats[key] = pd.DataFrame(columns=cols)

    # Merge precomputed statistics
    df = df.merge(train_stats["item_popularity"], on="itemid", how="left")
    df["item_popularity"] = df["item_popularity"].fillna(0)

    df = df.merge(train_stats["item_conversion_rate"], on="itemid", how="left")
    df["item_conversion_rate"] = df["item_conversion_rate"].fillna(0)

    df = df.merge(train_stats["category_popularity"], on="category_id", how="left")
    df["category_popularity"] = df["category_popularity"].fillna(0)

    df = df.merge(train_stats["category_conversion_rate"], on="category_id", how="left")
    df["category_conversion_rate"] = df["category_conversion_rate"].fillna(0)

    df = df.merge(train_stats["user_activity"], on="visitorid", how="left")
    df["user_event_count"] = df["user_event_count"].fillna(0)

    df = df.merge(train_stats["user_purchase_rate"], on="visitorid", how="left")
    df["user_purchase_rate"] = df["user_purchase_rate"].fillna(0)

    # Sort by timestamp for sequential features
    df = df.sort_values("timestamp")

    # Sequential interaction features (cumulative counts)
    df["user_item_interaction_count"] = (
        df.groupby(["visitorid", "itemid"]).cumcount()
    )

    df["user_category_affinity"] = (
        df.groupby(["visitorid", "category_id"]).cumcount()
    )

    # Recent item popularity
    df = df.merge(train_stats["recent_item_popularity"], on="itemid", how="left")
    df["recent_item_popularity"] = df["recent_item_popularity"].fillna(0)

    # Item popularity ratio (how popular is this item compared to its category average?)
    category_avg_popularity = (
        train_stats["item_popularity_for_ratio"]
        .merge(df[["itemid", "category_id"]].drop_duplicates(), on="itemid")
        .groupby("category_id")["item_popularity"]
        .mean()
        .reset_index()
        .rename(columns={"item_popularity": "category_avg_item_popularity"})
    )

    df = df.merge(category_avg_popularity, on="category_id", how="left")
    df["item_popularity_ratio"] = (
        df["item_popularity"] / (df["category_avg_item_popularity"] + 1)
    ).fillna(0)

    # Select final feature set
    df = df[
        [
            "visitorid",
            "itemid",
            "category_id",
            "hour",
            "dayofweek",
            "is_weekend",
            "is_evening",
            "is_working_hours",
            "item_popularity",
            "item_conversion_rate",
            "recent_item_popularity",
            "item_popularity_ratio",
            "category_popularity",
            "category_conversion_rate",
            "user_event_count",
            "user_purchase_rate",
            "user_item_interaction_count",
            "user_category_affinity",
            "label",
            "timestamp",
        ]
    ]

    if is_train:
        return df, train_stats
    else:
        return df


In [8]:
train_df, train_stats = load_and_clean_events(
    path,
    cat_df,
    is_train=True
)


In [9]:
train_df[train_df['label']==1]

,visitorid,itemid,category_id,hour,dayofweek,is_weekend,is_evening,is_working_hours,item_popularity,item_conversion_rate,recent_item_popularity,item_popularity_ratio,category_popularity,category_conversion_rate,user_event_count,user_purchase_rate,user_item_interaction_count,user_category_affinity,label,timestamp
466,869008,40685,1542,3,6,1,0,0,12,0.083333,0.0,0.814520,13815,0.021281,3,0.333333,2,2,1,2015-05-03 03:27:21.391
644,345781,438400,-1,3,6,1,0,0,15,0.066667,0.0,2.446709,255585,0.001858,8,0.125000,3,3,1,2015-05-03 03:35:01.772
1159,586756,440917,1415,4,6,1,0,0,47,0.042553,0.0,1.259325,6225,0.012851,6,0.166667,3,3,1,2015-05-03 04:01:47.591
1269,435495,175893,1244,4,6,1,0,0,6,0.166667,0.0,0.569138,10656,0.011167,3,0.333333,2,2,1,2015-05-03 04:07:38.961
1665,266417,445106,470,4,6,1,0,0,95,0.042105,0.0,2.798303,1944,0.026235,75,0.026667,2,4,1,2015-05-03 04:31:14.903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964546,152963,12504,1493,2,4,0,0,0,8,0.125000,1.0,1.063255,4880,0.031762,3217,0.115947,0,6,1,2015-09-18 02:08:16.512
2964547,152963,72462,1493,2,4,0,0,0,2,0.500000,1.0,0.265814,4880,0.031762,3217,0.115947,0,7,1,2015-09-18 02:08:16.512
2964548,152963,380196,973,2,4,0,0,0,36,0.055556,9.0,1.442326,5942,0.007910,3217,0.115947,4,11,1,2015-09-18 02:08:16.528
2964821,152963,362697,1663,2,4,0,0,0,82,0.036585,6.0,8.696203,5791,0.005180,3217,0.115947,2,7,1,2015-09-18 02:38:18.098


In [10]:
train_df['visitorid'].value_counts().sort_values(ascending=False)

visitorid
1150086    8148
530559     4497
152963     3217
895999     2618
163561     2592
           ... 
1207677       1
624573        1
998517        1
525390        1
485144        1
Name: count, Length: 1407580, dtype: int64